# Init

In [47]:
!pip install gensim

In [59]:
# geral
import pickle
import pandas as pd

# gensim
import gensim.corpora as corpora

# sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
with open('pickle/df_proc.p', 'rb') as file:
    df = pickle.load(file)
df.head()

,text
0,programa treinamento jornalismo agroindústria ...
1,ofício enviado sergio presidente michel esquiv...
2,governador sérgio cabral transferido complexo ...
3,aplausos federal sergio responsável operação i...
4,executivo empreiteira odebrecht afirmou acordo...


In [50]:
with open('pickle/optimal_model.p', 'rb') as file:
    optimal_model = pickle.load(file)

# Anexar classificação de tópicos no dataset

In [51]:
tokenized_docs = [doc.lower().split() for doc in df['text']]
id2word = corpora.Dictionary(tokenized_docs)
corpus = [id2word.doc2bow(text) for text in tokenized_docs]

In [52]:
def get_document_topic(lda_model, corpus):
    document_topics = []
    for bow in corpus:
        topic_probs = lda_model.get_document_topics(bow, minimum_probability=0.0)
        dominant_topic = max(topic_probs, key=lambda x: x[1])[0]
        document_topics.append(dominant_topic)
    return document_topics

In [53]:
target = get_document_topic(optimal_model, corpus)

In [54]:
tfidf = TfidfVectorizer(
    strip_accents='ascii',
    min_df = 0.1,
    max_features = 5000
)

tfidf_matrix = tfidf.fit_transform(df['text'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

In [55]:
tfidf_df['target'] = target

In [56]:
tfidf_df

,acordo,afirma,afirmou,apesar,brasil,brasileira,brasileiro,campanha,centro,chegou,...,regiao,resultado,rousseff,seguranca,semana,situacao,trabalho,ultima,ultimos,target
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.113346,0.000000,0.000000,0.0,0.0,6
1,0.000000,0.000000,0.000000,0.218331,0.000000,0.0,0.000000,0.224543,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,3
2,0.106243,0.000000,0.198091,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.135876,0.000000,0.0,0.0,6
3,0.000000,0.000000,0.403944,0.000000,0.262744,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.139283,0.0,0.113967,0.000000,0.000000,0.0,0.0,5
4,0.234460,0.100607,0.174861,0.000000,0.000000,0.0,0.000000,0.462245,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10025,0.000000,0.000000,0.000000,0.000000,0.447752,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,9
10026,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.432977,0.000000,0.000000,0.152649,...,0.0,0.163918,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,2
10027,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,7
10028,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,3


# Process

In [63]:
with open('pickle/tfidf_df.p', 'wb') as file:
    pickle.dump(tfidf_df, file)

In [57]:
X = tfidf_df.drop(columns=['target'])
y = tfidf_df['target']

In [60]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)